Carlos Bravo Garrán - 100474964

# __Seed Clustering__

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.cm as cm
style.use('ggplot') or plt.style.use('ggplot')

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist




import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### __1. Load the dataset__

Load the seed dataset from a CSV file. 

The features are stored in `X`, and the target class labels are stored in `y`.

Add seed variable for random states (`100474964`)


In [ ]:
data = pd.read_csv('data/semillas.csv')
X = data.drop(columns=['clase'])
y = data['clase']

seed = 100474964

print(data.head())

### __2. Comparison of Scalers__

This section identifies the most appropriate scaler for the seed dataset before applying clustering algorithms. Scaling ensures that all features contribute equally to the distance calculations.

Three scalers are compared:
- MinMaxScaler
- RobustScaler
- StandardScaler

The scaled data is projected into 2D using PCA for visual evaluation.

In [ ]:
scalers = {
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler(),
    'StandardScaler': StandardScaler()
}

plt.figure(figsize=(18, 5))

for i, (name, scaler) in enumerate(scalers.items(), 1):
    pipeline = make_pipeline(scaler, PCA(n_components=2, random_state=seed))
    
    X_pca = pipeline.fit_transform(X)
    
    plt.subplot(1, 3, i)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.7, s=50)
    plt.title(f'{name} + PCA')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.grid(True)

plt.suptitle('Comparison of Scalers with PCA (By Seed Class)', fontsize=16)
plt.tight_layout()
plt.show()


#### 2.1 Best Scaler Selection

After observing the PCA plots:

- **MinMaxScaler**: The data points are well distributed with moderate separation between different seed classes. Although some overlap exists, the distribution appears balanced and suitable for clustering.
- **RobustScaler**: There is a good separation of classes, but the spread of the data is larger, which may not be ideal for density-based methods.
- **StandardScaler**: While resistant to outliers, the classes are not well separated, making clustering more difficult.

The scaler selected is **MinMaxScaler** because it provides a balanced and homogeneous distribution of the data, facilitating the identification of clusters without introducing large variations in scale.


#### 2.2 Variance Explained by PCA

To ensure that the 2D projection using PCA retains enough information from the original dataset, calculate, after applying each scaler, the variance explained by the two principal components and the total variance.


In [ ]:
variance_ratios = {}

for name, scaler in scalers.items():
    X_scaled = scaler.fit_transform(X)
    pca = PCA(n_components=2, random_state=seed)
    X_pca = pca.fit_transform(X_scaled)
    
    variance_explained = np.sum(pca.explained_variance_ratio_)
    variance_ratios[name] = {
        'PC1': pca.explained_variance_ratio_[0],
        'PC2': pca.explained_variance_ratio_[1],
        'Total': variance_explained
    }

variance_table = pd.DataFrame.from_dict(variance_ratios, orient='index')
variance_table.index.name = 'Scaler'
variance_table.reset_index(inplace=True)

display(variance_table)


The results show that all three scalers achieve a high variance explanation (>85%), indicating that the 2D PCA projection is representative of the original dataset in every case.

Among them, **MinMaxScaler** achieves the highest variance explained, with 91.81%.

It's confirmed that using **MinMaxScaler** is appropriate, as it retains the largest proportion of the original data variance after dimensionality reduction. Therefore, MinMaxScaler is selected for the clustering tasks.

#### 2.3 Scaling and PCA
The dataset was scaled using the selecter scaler (**MinMaxScaler**) and reduced to two dimensions using PCA

In [ ]:
pipeline = make_pipeline(
    MinMaxScaler(),
    PCA(n_components=2, random_state=seed)
)

X_final_pca = pipeline.fit_transform(X)

plt.figure(figsize=(4, 3))
plt.scatter(X_final_pca[:, 0], X_final_pca[:, 1], c=y, cmap='viridis', alpha=0.7)
plt.title('MinMaxScaler + PCA (Final Preprocessing)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('MinMaxScaler + PCA', fontsize=10)
plt.grid(True)
plt.show()


### __3. Clustering__

Apply unsupervised clustering techniques on the PCA-transformed seed dataset to identify natural groupings:

- K-Means
- Hierarchical Clustering
- DBSCAN

For each method,the key hyperparameters are tuned, the results visualized and their performance compared to determine which best captures the structure of the data.


#### __3.1 K-means clustering__

Based on the visual inspection of the PCA plot, k = 3 is initially selected as a reasonable number of clusters.

##### __Model creation and training__

In [ ]:
modelo_kmeans = KMeans(
    n_clusters=3,
    n_init=25,
    random_state=seed
)

modelo_kmeans.fit(X_final_pca)


##### __Cluster prediction__

In [ ]:
y_pred = modelo_kmeans.predict(X_final_pca)


##### __Cluster Visualization and Evaluation__


In [ ]:
colors = ['yellow', 'purple', 'green', 'orange', 'purple']

fig, ax = plt.subplots(1, 1, figsize=(6, 5))

for cluster in np.unique(y_pred):
    ax.scatter(
        x = X_final_pca[y_pred == cluster, 0],
        y = X_final_pca[y_pred == cluster, 1],
        color = colors[cluster],
        label = f"Cluster {cluster}",
        edgecolor = 'black',
        marker = 'o'
    )

ax.scatter(
    x = modelo_kmeans.cluster_centers_[:, 0],
    y = modelo_kmeans.cluster_centers_[:, 1],
    c = 'red',
    s = 200,
    marker = '*',
    label = 'Centroids'
)

ax.set_title('Clusters generated by KMeans (K=3)')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.legend()
ax.grid(True)
plt.show()


The K-Means clustering with k = 3 produced three clearly defined groups in the PCA-transformed space.

- Each cluster is compact and well-separated from the others, with minimal overlap between groups.
- The centroids are located close to the center of each cluster, indicating that the algorithm has correctly captured the dense regions of the data.
- The overall structure suggests that the data naturally forms three main groups, supporting the choice of k = 3.
- Although the clusters are not perfectly spherical, the distribution around each centroid is balanced and coherent.

In conclusion, the visual inspection confirms that K-Means with k = 3 is a good fit for the structure observed in the seed dataset.

Even so, other values of k will be tested to ensure robustness.

#### __Evaluation of K-Means with Different Values of k__

In this section, the performance of the K-Means algorithm is evaluated using different values of k (number of clusters). This helps identify the optimal number of clusters that best represents the structure of the data.

Plots are generated for k = 2 and k = 4, comparing the distribution of data in the PCA space.


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Results for K = 2
y_predict_2 = KMeans(n_clusters=2, n_init=25, random_state=seed).fit_predict(X=X_final_pca)
ax[0].scatter(
    x = X_final_pca[:, 0],
    y = X_final_pca[:, 1],
    c = y_predict_2,
    marker = 'o',
    edgecolor = 'black'
)
ax[0].set_title('KMeans with K=2')
ax[0].set_xlabel('PC1')
ax[0].set_ylabel('PC2')
ax[0].grid(True)

# Results for K = 4
y_predict_4 = KMeans(n_clusters=4, n_init=25, random_state=seed).fit_predict(X=X_final_pca)
ax[1].scatter(
    x = X_final_pca[:, 0],
    y = X_final_pca[:, 1],
    c = y_predict_4,
    marker = 'o',
    edgecolor = 'black'
)
ax[1].set_title('KMeans with K=4')
ax[1].set_xlabel('PC1')
ax[1].set_ylabel('PC2')
ax[1].grid(True)

plt.tight_layout()
plt.show()



- With **k = 2**, the data is divided into two large groups, losing important internal structures.
- With **k = 4**, the clustering tends to oversegment the data, creating small clusters that do not correspond to natural groupings.
- With **k = 3**, the data is divided into three compact and well-separated groups, without significant overlap or oversegmentation.

The visual comparison confirms that **k = 3** is the most appropriate number of clusters for this dataset.

##### __Elbow Method__

To support the selection of the number of clusters, the elbow method is applied, which analyzes how the inertia decreases as the number of clusters increases.

The optimal number of clusters is identified at the point where adding an additional cluster no longer significantly reduces the inertia.

In [ ]:
inertia = []
K = range(1, 11)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X_final_pca)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(6, 4))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()


The plot of inertia facing the number of clusters shows a clear "elbow" at k = 3, reconfirming that k = 3 is the most appropriate number of clusters for this dataset.

##### __Silhouette Method__

The silhouette coefficient measures how similar points within the same cluster are compared to those in other clusters. The closer to 1, the better defined the cluster.

We will calculate and visualize the average silhouette coefficient for different values of k (from 2 to 10):

In [ ]:
silhouette_scores = []
K = range(2, 11)

for k in K:
    kmeans = KMeans(n_clusters=k, n_init=25, random_state=seed)
    labels = kmeans.fit_predict(X_final_pca)
    score = silhouette_score(X_final_pca, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(6, 4))
plt.plot(K, silhouette_scores, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Average Silhouette Score')
plt.title('Silhouette Method for Optimal k')
plt.grid(True)
plt.show()


It can be observed that k = 2 has the highest silhouette coefficient (~0.56), suggesting good separation between two groups. However, this may be because the classes are divided into two large generic groups, ignoring more natural subdivisions.

This differs from the choice we made earlier of k = 3, so to clarify, we will generate individual silhouette plots for k = 2 to k = 5 to see if the clusters are well-defined and how consistent the assignments are.

In [ ]:
Ks = [2, 3, 4, 5]
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, k in enumerate(Ks):
    ax = axes[idx]
    kmeans = KMeans(n_clusters=k, n_init=25, random_state=seed)
    cluster_labels = kmeans.fit_predict(X_final_pca)

    silhouette_vals = silhouette_samples(X_final_pca, cluster_labels)

    y_lower = 10
    for i in range(k):
        ith_cluster_silhouette_vals = silhouette_vals[cluster_labels == i]
        ith_cluster_silhouette_vals.sort()

        size_cluster_i = ith_cluster_silhouette_vals.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / k)
        ax.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_vals,
            facecolor=color,
            edgecolor=color,
            alpha=0.7
        )

        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10 

    avg_score = silhouette_score(X_final_pca, cluster_labels)
    ax.text(0.7, 0.9, f'Avg Silhouette: {avg_score:.3f}', transform=ax.transAxes, color="red", fontsize=10)


    ax.axvline(x=avg_score, color="red", linestyle="--")
    ax.set_title(f'Silhouette plot for K = {k}')
    ax.set_xlabel('Silhouette coefficient values')
    ax.set_ylabel('Cluster label')
    ax.set_yticks([])
    ax.set_xlim([-0.1, 1])
    ax.grid(True)

plt.tight_layout()
plt.show()


| Number of Clusters (k) | Average Silhouette | Analysis                                                                 |
|------------------------|--------------------|-------------------------------------------------------------------------------|
| 2                      | ~0.56           | Clear and compact structure, but may be an excessive simplification          |
| 3                      | ~0.51           | Good separation, consistent with the actual number of classes (3)            |
| 4                      | ~0.43           | Overclustering, appearance of small clusters and overlap between groups      |
| 5                      | ~0.40           | Worse cohesion, some artificial clusters and poor point assignment           |


The silhouette plots show that the values of n_clusters = 4 and 5 are a poor choice due to below-average silhouette scores and the appearance of clusters with very unequal sizes.

On the other hand, the silhouette analysis is more ambiguous when comparing k = 2 and k = 3, as we have seen before. The value of k = 2 presents the highest average silhouette coefficient (~0.56), and its clusters are clearly separated, although the grouping may be too general, hiding more detailed structures. This can also be seen in the plot, where one of the clusters (cluster 0) is larger and encompasses multiple subsets.

With k = 3, the average silhouette coefficient is slightly lower (~0.51), but the three clusters exhibit good internal cohesion and relative separation, with more balanced sizes and consistent bars. Additionally, k = 3 is consistent with the results obtained previously.

In summary, although k = 2 achieves the highest average score, the visual and conceptual analysis suggests that k = 3 offers a richer, more interpretable segmentation aligned with the actual structure of the dataset.



#### __3.2 Hierarchical Clustering__

In this section, **hierarchical clustering** (Agglomerative Clustering) is applied to the seed dataset transformed using PCA. Different linkage methods are tested (`linkage`), and the results are analyzed both visually and quantitatively through scatter plots, dendrograms, and the silhouette coefficient.

__Tested Linkage Methods__

The following hierarchical linkage methods are compared:
- **Ward**: Minimizes variance within clusters.
- **Complete**: Maximizes the distance between the farthest points.
- **Average**: Uses the average distance between points.
- **Single**: Considers the minimum distance between points (may form chains).


##### **Comparison of Methods Using Scatter Plots**

In [ ]:
linkage_methods = ['ward', 'complete', 'average', 'single']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, method in enumerate(linkage_methods):
    # Ward sólo admite distancia euclídea
    if method == 'ward':
        model = AgglomerativeClustering(n_clusters=3, linkage=method)
    else:
        model = AgglomerativeClustering(n_clusters=3, linkage=method, metric='euclidean')

    y_pred = model.fit_predict(X_final_pca)
    score = silhouette_score(X_final_pca, y_pred)
    print(f"Silhouette Score ({method} linkage): {score:.3f}")

    axes[i].scatter(X_final_pca[:, 0], X_final_pca[:, 1], c=y_pred, cmap='viridis', edgecolor='k', alpha=0.7)
    axes[i].set_title(f'Linkage: {method}')
    axes[i].set_xlabel('PC1')
    axes[i].set_ylabel('PC2')
    axes[i].grid(True)

plt.suptitle('Agglomerative Clustering with Different Linkage Methods', fontsize=16)
plt.tight_layout()
plt.show()


Los resultados muestran que el método `ward` genera agrupaciones compactas y bien separadas, con una puntuación de silueta ligeramente superior a los demás. Los métodos `average` y `complete` ofrecen buenos resultados, pero con mayor solapamiento. Por otro lado, el método `single` suele ser menos recomendable, ya que tiende a formar cadenas y clusters poco definidos, por lo que este método se descarta directamente.


Function to create a dendrogram from the hierarchical clustering model.


In [ ]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    dendrogram(linkage_matrix, **kwargs)

Create hierarchical models with different linkage methods and construct the complete clustering tree without fixing the number of clusters, to generate dendrograms with the full structure of merges.

In [ ]:

modelo_hclust_complete = AgglomerativeClustering(
    linkage  = 'complete',
    distance_threshold = 0,
    n_clusters         = None
)
modelo_hclust_complete.fit(X=X_final_pca)

modelo_hclust_average = AgglomerativeClustering(
    linkage  = 'average',
    distance_threshold = 0,
    n_clusters         = None
)
modelo_hclust_average.fit(X=X_final_pca)

modelo_hclust_ward = AgglomerativeClustering(
    linkage  = 'ward',
    distance_threshold = 0,
    n_clusters         = None
)
modelo_hclust_ward.fit(X=X_final_pca)

##### **Visualization of Complete Dendrograms**

Complete dendrograms are generated for the `ward`, `average`, and `complete` methods, based on models trained without fixing the number of clusters (`n_clusters=None`) and with `distance_threshold=0`. These dendrograms display the entire hierarchy of merges between observations, from individual points to the total grouping.

This representation allows for the analysis of the global hierarchical structure and observation of the distances at which the merges occur.


In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(20, 20))

plot_dendrogram(modelo_hclust_average, labels=range(len(X_final_pca)), color_threshold=0, ax=axs[0])
axs[0].set_title("Linkage average")

plot_dendrogram(modelo_hclust_complete, labels=range(len(X_final_pca)), color_threshold=0, ax=axs[1])
axs[1].set_title("Linkage complete")

plot_dendrogram(modelo_hclust_ward, labels=range(len(X_final_pca)), color_threshold=0, ax=axs[2])
axs[2].set_title("Linkage ward")

plt.tight_layout();

Among the three methods, the dendrogram generated with **`ward`** stands out for presenting a more balanced structure with more significant distance jumps at the upper levels of the tree, suggesting clearer and more natural divisions in the dataset.

In contrast, the dendrograms with `average` and `complete` show a more gradual progression, without such defined cuts, which may make it harder to visually identify an optimal number of clusters.

Even so, we will truncate the dendrograms to view them more clearly.


##### __Truncated Dendrograms for Easier Interpretation__

Since the complete dendrograms are very extensive and difficult to read, **truncated** versions are also generated using `truncate_mode='level'` and `p=5`, which show only the last levels of the hierarchical clustering.

This approach focuses on the main divisions and facilitates the visual identification of relevant groupings, such as choosing a reasonable number of clusters based on the cutting height.

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(16, 12))

plot_dendrogram(modelo_hclust_average, labels=range(len(X_final_pca)), 
                truncate_mode='level', p=5, leaf_rotation=90, leaf_font_size=8, ax=axs[0])
axs[0].set_title("Linkage average")

plot_dendrogram(modelo_hclust_complete, labels=range(len(X_final_pca)), 
                truncate_mode='level', p=5, leaf_rotation=90, leaf_font_size=8, ax=axs[1])
axs[1].set_title("Linkage complete")

plot_dendrogram(modelo_hclust_ward, labels=range(len(X_final_pca)), 
                truncate_mode='level', p=5, leaf_rotation=90, leaf_font_size=8, ax=axs[2])
axs[2].set_title("Linkage ward")

plt.tight_layout()
plt.show()


It is confirmed that the **`ward`** method presents clearer and more well-defined divisions, with a final merge occurring at a significantly higher height, indicating greater separation between groups.

In comparison, the **`average`** and **`complete`** methods show more gradual and less abrupt groupings, suggesting that the differences between consecutive groups are smoother and more progressive.

Overall, this visualization supports the choice of `ward` as the most effective method for capturing the hierarchical structure of the dataset.

##### __Visual Cut of the Dendrogram__

To determine the number of clusters, the dendrogram is visually cut at a certain height. For the Ward method, which has shown the best results, a cut is made at height 5.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

altura_corte = 5

plot_dendrogram(
    modelo_hclust_ward,
    labels=range(len(X_final_pca)),
    truncate_mode='level',
    p=5,
    leaf_rotation=90,
    leaf_font_size=8,
    color_threshold=altura_corte,
    ax=ax
)

ax.set_title("Linkage: Ward — Height cut = 5")
ax.axhline(y=altura_corte, color='black', linestyle='--', label=f'Height cut = {altura_corte}')
ax.legend()
plt.tight_layout()
plt.show()


**3 main clusters** are clearly identified. This approach allows for intuitive segmentation of the data, based on the largest distance jumps between consecutive merges.

The cut at this height coincides with a point where a significant separation occurs in the hierarchical tree, suggesting that the natural structure of the data favors the existence of three large groups. This observation reinforces the results obtained with other methods such as K-Means and the silhouette analysis for k = 3.

##### __Silhouette Coefficient Method__

Apply the silhouette coefficient method to find the optimal number of clusters in hierarchical clustering (linkage='ward').

In [ ]:
range_n_clusters = range(2, 15)
valores_medios_silhouette = []

for n_clusters in range_n_clusters:
    modelo = AgglomerativeClustering(
        linkage    = 'ward',
        n_clusters = n_clusters
    )

    cluster_labels = modelo.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    valores_medios_silhouette.append(silhouette_avg)

fig, ax = plt.subplots(1, 1, figsize=(6, 3.84))
ax.plot(range_n_clusters, valores_medios_silhouette, marker='o')
ax.set_title("Mean evolution of silhouette indexes")
ax.set_xlabel('Clusters name')
ax.set_ylabel('Siluette index mean');

The silhouette index analysis shows that the maximum value is reached for k = 2, with an average coefficient close to 0.46. From that point, the index decreases steadily, suggesting that dividing the dataset into more groups reduces cohesion within each cluster and separation between them.

Although this result suggests that k = 2 is optimal from a purely mathematical perspective, this value may be too simplistic regarding the actual structure of the dataset, which originally includes three seed classes. Therefore, k = 3 or k = 4 are also considered reasonable options, especially if a more informative segmentation aligned with prior knowledge is sought.


##### __Choice of Method and Number of Clusters__

After comparing different linkage methods and analyzing the results both visually and quantitatively, it is concluded that the best hierarchical clustering method for this dataset is the **ward** method, as it provides the most compact and well-separated clusters, supported by a good silhouette index.

The visual cut of the dendrogram at height 5 generates **3 clearly differentiated clusters**, which align with the most coherent results from the K-Means analysis and adequately reflect the structure of the dataset.

Therefore, we select the **ward** method and a number of clusters equal to 3 as the most suitable configuration to represent the natural groupings of seeds in this problem.


In [ ]:
modelo_hclust_ward = AgglomerativeClustering(
    linkage  = 'ward',
    n_clusters = 3
)
modelo_hclust_ward.fit(X=X_scaled)